# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import google_api_key
gmaps.configure(api_key=google_api_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
city_data_file = "citydataclean.csv"

# Read the city data
city_data_clean = pd.read_csv(city_data_file)

# Display the data table for preview
city_data_clean.head()

,City,Country,City ID,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Belaya Gora,RU,2126785,68.5333,146.4167,-22.45,73,0,4.36,1616111022
1,Provideniya,RU,4031574,64.3833,-173.3000,-0.40,60,0,2.24,1616111023
2,East London,ZA,1006984,-33.0153,27.9116,71.82,86,3,12.15,1616111024
3,Mossamedes,AO,3347019,-15.1961,12.1522,72.68,73,81,6.24,1616111025
4,Port Hedland,AU,2063042,-20.3167,118.5667,77.00,69,90,14.97,1616111026


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID", center=(9.0778, 8.6775), zoom_level=1.8)

# Store latitude and longitude in locations
locations = city_data_clean[["Lat", "Long"]]

# Fill NaN values and convert to float
humidity = city_data_clean["Humidity"].astype(float)
max_humidity = city_data_clean["Humidity"].max()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
temp_min = 83
temp_max = 87
windspeed_max = 10
windspeed_min = 4
cloudiness_max = 30

ideal_cities_df = city_data_clean.loc[((city_data_clean['Max Temp']>=temp_min) & 
                                   (city_data_clean['Max Temp']<=temp_max) & 
                                   (city_data_clean['Cloudiness']<=cloudiness_max)& 
                                   (city_data_clean['Wind Speed']>=windspeed_min)& 
                                   (city_data_clean['Wind Speed']<=windspeed_max)),:
                                  ].copy()


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
ideal_cities_df

,City,Country,City ID,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
128,La Gomera,GT,3594575,14.0833,-91.0500,86.00,48,1,6.91,1616111135
316,Ures,MX,3980782,29.4333,-110.4000,84.99,10,24,5.99,1616111300
415,Kavaratti,IN,1267390,10.5669,72.6420,83.34,67,11,6.93,1616111376
429,Ambon City,ID,1651531,-3.6954,128.1814,84.20,79,20,4.61,1616111387
532,Massaguet,TD,2428231,12.4755,15.4365,86.00,21,20,4.61,1616111480
676,Solenzo,BF,2355404,12.1833,-4.0833,83.57,14,18,6.13,1616111613
916,Pochutla,MX,3517970,15.7432,-96.4661,86.00,45,20,8.05,1616111800


In [23]:
#Add blank column to store hotel
ideal_cities_df['Hotel'] = ""

#join lat-long into one column to simplify location lookup
ideal_cities_df['Lat-Long'] = ideal_cities_df["Lat"].astype(str) +','+ ideal_cities_df["Long"].astype(str)

In [24]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "lodging",
    "key": google_api_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in ideal_cities_df.iterrows():

    # get restaurant type from df
    location = row['Lat-Long']

    # add keyword to params dict
    params['location'] = location

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {location}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        ideal_cities_df.loc[index, 'Hotel'] = results[0]['name']

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        ideal_cities_df.loc[index, 'Hotel'] = "no hotel found"
        
    print("------------")

Closest hotel is Cede.E.T.V.
------------
Closest hotel is Hotel HACIENDA San Francisco.
------------
Closest hotel is Poddar Sworone.
------------
Closest hotel is Airy Lapangan Merdeka Ambon Wem Reawaru 7B.
------------
Closest hotel is Alwatane.
------------
Missing field/result... skipping.
------------
Closest hotel is Hotel Posada San Jose.
------------


In [25]:
#review dataframe
ideal_cities_df

,City,Country,City ID,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel,Lat-Long
128,La Gomera,GT,3594575,14.0833,-91.0500,86.00,48,1,6.91,1616111135,Cede.E.T.V,"14.0833,-91.05"
316,Ures,MX,3980782,29.4333,-110.4000,84.99,10,24,5.99,1616111300,Hotel HACIENDA San Francisco,"29.4333,-110.4"
415,Kavaratti,IN,1267390,10.5669,72.6420,83.34,67,11,6.93,1616111376,Poddar Sworone,"10.5669,72.642"
429,Ambon City,ID,1651531,-3.6954,128.1814,84.20,79,20,4.61,1616111387,Airy Lapangan Merdeka Ambon Wem Reawaru 7B,"-3.6954,128.1814"
532,Massaguet,TD,2428231,12.4755,15.4365,86.00,21,20,4.61,1616111480,Alwatane,"12.4755,15.4365"
676,Solenzo,BF,2355404,12.1833,-4.0833,83.57,14,18,6.13,1616111613,no hotel found,"12.1833,-4.0833"
916,Pochutla,MX,3517970,15.7432,-96.4661,86.00,45,20,8.05,1616111800,Hotel Posada San Jose,"15.7432,-96.4661"


In [26]:
#clean dataframe to remove any cities where no hotels were found or other null values

ideal_cities_df = ideal_cities_df.loc[ideal_cities_df["Hotel"]!= "no hotel found"]
ideal_cities_df = ideal_cities_df.dropna(how='any')

In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_map = [info_box_template.format(**row) for index, row in ideal_cities_df.iterrows()]
locations = ideal_cities_df[['Lat','Long']]

In [29]:
# Add marker layer to existing map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_map)
fig.add_layer(markers)
# Display final figure
fig

Figure(layout=FigureLayout(height='420px'))